# Get a raw image, display it and overlay the position of the reference sources

In [1]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display.ds9 as ds9
import numpy

butler = dafPersist.Butler("/sps/lsst/dataproducts/desc/DC2/Run1.2p/w_2018_15/work/0/input")
dataid = {'visit':286254, 'raft':'2,2', 'sensor':'1,1'}

raw = butler.get('eimage', dataid)
wcsRaw = raw.getWcs()

In [2]:
# Display image in ds9 (the display_ds9 stack package should be set up)
ds9.mtv(raw.getMaskedImage().getImage(),frame=1,wcs=wcsRaw)

ds9 doesn't appear to be running (XPA: 
  File "python/lsst/display/ds9/xpa.cc", line 115, in const char* lsst::display::ds9::{anonymous}::XPASet1(XPA, char*, char*, char*, char*, int)
    XPASet returned 0 {0}
lsst::pex::exceptions::IoError: 'XPASet returned 0'
, (;iconify no; raise)), I'll try to exec it for you


RuntimeError: 
  File "python/lsst/afw/display/simpleFits.cc", line 441, in void lsst::afw::display::writeBasicFits(int, const ImageT&, const lsst::afw::geom::SkyWcs*, const char*) [with ImageT = lsst::afw::image::Image<float>]
    Error writing data for row 12 {0}
lsst::pex::exceptions::RuntimeError: 'Error writing data for row 12'


## Create and load a catalog with reference objects 

In [21]:
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage
from lsst.meas.algorithms import LoadIndexedReferenceObjectsTask

# Configure LoadIndexedReferenceObjectsTask
refConfig = LoadIndexedReferenceObjectsTask.ConfigClass()
refConfig.ref_dataset_name = 'cal_ref_cat'
refTask = LoadIndexedReferenceObjectsTask(butler, config=refConfig)

#define a reference filter (not critical for what we are doing) 
filter = 'lsst_z_smeared'

# Find the approximate celestial coordinates of the sensor's center 
centerPixel = afwGeom.Point2D(2000., 2000.)
centerCoord = wcsRaw.pixelToSky(centerPixel)

# Retrieve reference object within a 0.5 deg radius circle around the sensor's center
radius = afwGeom.Angle(0.5, afwGeom.degrees)
ref = refTask.loadSkyCircle(centerCoord, radius, filter).refCat.copy(deep=True)

In [12]:
# We can now display the reference catalog as an astropy table
ref.asAstropy()

id,coord_ra,coord_dec,parent,lsst_u_smeared_flux,lsst_g_smeared_flux,lsst_r_smeared_flux,lsst_i_smeared_flux,lsst_z_smeared_flux,lsst_y_smeared_flux,lsst_u_smeared_fluxSigma,lsst_g_smeared_fluxSigma,lsst_r_smeared_fluxSigma,lsst_i_smeared_fluxSigma,lsst_z_smeared_fluxSigma,lsst_y_smeared_fluxSigma,resolved,variable,raJ2000,decJ2000,sigma_raJ2000,sigma_decJ2000,lsst_u,lsst_g,lsst_r,lsst_i,lsst_z,lsst_y,properMotionRa,properMotionDec,parallax,radialVelocity,centroid_x,centroid_y,hasCentroid
,rad,rad,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool
1606134359044,0.9761448726930843,-0.49879445646568804,-1,9.7796750549775e-10,7.8275080136769e-09,2.7281610293284745e-08,1.362362751142661e-07,3.004945563663417e-07,5.050308790040715e-07,9.007413598366767e-13,7.209401307033546e-12,2.5127291669676152e-11,1.2547824648125792e-10,2.76765714405999e-10,4.6515062939858195e-10,False,False,55.928981400000005,-28.5788172,2.777777778e-08,2.777777778e-08,31.42463871,29.16652258,27.81029679,26.06240314,25.20645634,24.64201952,0.0,0.0,0.0,0.0,nan,nan,False
1606135098372,0.9760801646406457,-0.4988508864930971,-1,2.380769032672208e-08,2.0858747212388522e-07,6.287617889230971e-07,1.4975035924346757e-06,2.2165321599365687e-06,2.6832905979097882e-06,2.1927693137971524e-11,1.921161615591077e-10,5.79111008887437e-10,1.3792517794580462e-09,2.041501563844735e-09,2.4714019723672638e-09,False,False,55.9252739,-28.582050400000004,2.777777778e-08,2.777777778e-08,27.95726445,25.60106319,24.40357973,23.46202805,23.03629847,22.82897694,0.0,0.0,0.0,0.0,nan,nan,False
1606137046020,0.9764270350326761,-0.49892901101818415,-1,2.743233280359078e-08,2.509410719890923e-07,7.51731465262389e-07,2.023425546923796e-06,3.14274086653614e-06,3.859631559693322e-06,2.5266112231839877e-11,2.3112526863281187e-10,6.923702663390994e-10,1.8636438004520227e-09,2.8945713081717225e-09,3.5548520375196807e-09,False,False,55.94514809999999,-28.586526599999996,2.777777778e-08,2.777777778e-08,27.80466487,25.4003179,24.20959971,23.13412846,22.65841326,22.43346687,0.0,0.0,0.0,0.0,nan,nan,False
470358356996,0.976237227338166,-0.4985601129385283,-1,9.20742446914302e-06,8.479273601539201e-05,0.0002693612066694988,0.0005353405846833931,0.0007352100531801463,0.0008628386226957041,8.480351331006933e-09,7.80969959772884e-08,2.4809083964323054e-07,4.930668999866791e-07,6.771534834687859e-07,7.947037401114569e-07,False,False,55.93427290000001,-28.5653903,2.777777778e-08,2.777777778e-08,21.48760696,19.07972397,17.8251242,17.07749389,16.73317839,16.55908591,0.0,0.0,0.0,0.0,nan,nan,False
1606132401156,0.9762940095107255,-0.49864995917498595,-1,1.1391037587257627e-09,9.951252565996098e-09,2.8870134881510275e-08,5.204015925249689e-08,6.770655313495656e-08,7.815296532368903e-08,1.049153333686171e-12,9.165442326032545e-12,2.6590376884357197e-11,4.793075797273421e-11,6.236003997862412e-11,7.198154117104278e-11,False,False,55.9375263,-28.5705381,2.777777778e-08,2.777777778e-08,31.25900304,28.90535806,27.74811621,27.10910271,26.82282837,26.66871621,0.0,0.0,0.0,0.0,nan,nan,False
470358406148,0.9762403681272493,-0.4985947951671724,-1,2.8583283242324602e-09,1.0051796714202432e-08,3.408738807145512e-08,2.104657507787589e-07,5.866005703566356e-07,1.2657035812306833e-06,2.6326176761041258e-12,9.258046908771621e-12,3.139564465297425e-11,1.9384612013158812e-10,5.402790915379977e-10,1.1657560793163801e-09,False,False,55.9344529,-28.5673775,2.777777778e-08,2.777777778e-08,30.26008143,28.89472014,27.56894369,25.59169889,24.47974747,23.64404219,0.0,0.0,0.0,0.0,nan,nan,False
1277256370180,0.9762286211202325,-0.49881306915347484,-1,2.6129089767307176e-07,2.6665460301505424e-06,7.712590993771333e-06,2.8597836634161998e-05,4.9446222287062604e-05,6.355067692621254e-05,2.4065781036681907e

## Display the bright reference sources as a small circle overlaid on the ds9 image

In [14]:
for r in ref:
    ra = r.get("coord_ra")
    dec = r.get("coord_dec")
    mag = r.get("lsst_z")
    c = afwCoord.IcrsCoord(ra, dec)
    pt = wcsRaw.skyToPixel(c)
    # Ignore sources which lye outside the sensor footprint or which are too faint 
    if (pt.getX() > 4000) or (pt.getX()<0) or (pt.getY()>4000) or (pt.getY()<0) or mag > 21:
        continue
    with ds9.Buffering():
        ds9.dot('o', pt.getX(), pt.getY(), frame=1, size=8)

### Note that reference sources does not exactly match the sources in the raw images according to Scott 
### this is due to the fact that PhoSim  does not include an internal model for the motion of the earth

## We can now redo the same thing with the calibrated exposure (calexp)

In [22]:
# We need to get a new butler as calexp are in the output directory while eImage(raw) was in the input

butler = dafPersist.Butler("/sps/lsst/dataproducts/desc/DC2/Run1.2p/w_2018_15/work/0/output")
calexp = butler.get('calexp', dataid)
wcs = calexp.getWcs()

# display in another frame
ds9.mtv(calexp.getMaskedImage().getImage(),frame=2,wcs=wcs)

In [23]:
for r in ref:
    ra = r.get("coord_ra")
    dec = r.get("coord_dec")
    mag = r.get("lsst_z")
    c = afwCoord.IcrsCoord(ra, dec)
    pt = wcs.skyToPixel(c)
    # Ignore sources which lye outside the sensor footprint or which are too faint 
    if (pt.getX() > 4000) or (pt.getX()<0) or (pt.getY()>4000) or (pt.getY()<0) or mag > 21:
        continue
    with ds9.Buffering():
        ds9.dot('o', pt.getX(), pt.getY(), frame=2, size=8)